In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# Convertir ecuaciones minimo a máximo
def min_to_max(constraints,to_transform,coefficients):
  coefficients *= -1
  for indice in to_transform:
    constraints[indice] *= -1

# Eliminar igualdades
def remove_equalities(equalities,constraints):
  for index in equalities:
    pivot = np.argmin(constraints[index,1:]) + 1
    constraints[index] /= constraints[index,pivot]
    constraints[index,pivot] = 0
    for row in range(constraints.shape[0]):
      if row != index:
        constraints[row] += constraints[index]*constraints[row,pivot]
    constraints = np.delete(constraints,index,axis=0)
    return constraints

def read_csv(path):
  pass

def load_config(path):
  pass

In [40]:
# Automatizar interfaz
start = time.time()
n_slacks = 3
constraints = np.array([[15000,5,3],[15000,3,5],[10000,1,0]],dtype = "float")
equalities = []
to_transform = []

if equalities:
  constraints = remove_equalities(equalities,constraints)

if to_transform:
  min_to_max(constraints,to_transform)

coefficients = np.array([0,0.5,0.5,0,0,0])
base = [3,4,5]
values = np.concatenate([constraints,np.identity(n_slacks)],axis=1)
result = np.full(coefficients.shape,-1)
while(True):
  C_b = np.array(coefficients[base])
  result = C_b@values - coefficients # Completamos la tabla
  if (result[1:] >= 0).all():
    break
  # Determinamos que elementos entra y sale de la base y realizamos el cambio
  goes_in = 1 + np.argmin(result[1:])
  goes_out = np.argmin(np.where(values[:,goes_in] <= 0, np.inf, values[:,0]*1/values[:,goes_in]))
  base[goes_out] = goes_in

  # Seleccionamos el pivote, normalizamos la fila y hacemos combinación lineal para evitar que haya multiples variables de la base en una misma ecuación
  pivot = goes_out
  values[pivot]/=values[pivot,goes_in]
  for row in range(values.shape[0]):
    if row != pivot:
      values[row] -= values[pivot]*values[row,goes_in]
print(base)
print(values)
print(result)
print(f"Tiempo de ejecución: {time.time()-start}")

[1, 2, 5]
[[ 1.875e+03  1.000e+00  0.000e+00  3.125e-01 -1.875e-01  0.000e+00]
 [ 1.875e+03  0.000e+00  1.000e+00 -1.875e-01  3.125e-01  0.000e+00]
 [ 8.125e+03  0.000e+00  0.000e+00 -3.125e-01  1.875e-01  1.000e+00]]
[1.875e+03 0.000e+00 0.000e+00 6.250e-02 6.250e-02 0.000e+00]
Tiempo de ejecución: 0.0018815994262695312
